In [16]:
import numpy as np
import scipy as sp
from utility import M,F, benchmark_mapping, get_best_connected, invert, score, get_highest_out_degrees
rng = np.random.default_rng(0)

In [4]:
m,n = 100,500
H_M,M_positions = get_highest_out_degrees(M,n)
H_F,F_positions = get_highest_out_degrees(F,m)
H_M = H_M.todense()
H_F = H_F.todense()
H_F[np.arange(m),np.arange(m)] = 0
H_M[np.arange(n),np.arange(n)] = 0
np.count_nonzero(H_M)/n**2, np.count_nonzero(H_F)/m**2

(0.204576, 0.2318)

In [2]:
A=M.todense()
B=F.todense()
N=A.shape[0]
A[np.arange(N),np.arange(N)]=0
B[np.arange(N),np.arange(N)]=0

In [4]:
mapping = invert(benchmark_mapping)

In [52]:
def get_scores_by_vertex(G,H,mapping):

    m = G.shape[0]
    scores = np.minimum(G,H[mapping[:,None],mapping[None,:]])
    return scores.sum(axis=0)+scores.sum(axis=1)

def move_or_swap(G,H, mapping, u, scores_by_vertex, usage, penalty_gradient):

    m = G.shape[0]
    n = H.shape[0]
    f_u = mapping[u]

    row = G[u,:]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = G[:,u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    X = H[:,mapping[None,rnz]].reshape((n,len(rnz)))
    Y = H[mapping[cnz,None],:].reshape((len(cnz),n))

    base_scores = np.minimum(row[None,:],X).sum(axis=1) + np.minimum(col[:,None],Y).sum(axis=0)
    u_scores = base_scores[mapping]

    move_scores = base_scores - base_scores[f_u]
    move_scores -= penalty_gradient(usage) 
    move_scores += penalty_gradient(usage[f_u]-1)
    move_scores[u] = 0

    row = H[f_u,mapping]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = H[mapping,f_u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    X = G[:,rnz[None,:]].reshape((m,len(rnz)))
    Y = G[cnz[:,None],:].reshape((len(cnz),m))

    f_u_scores = np.minimum(row[None,:],X).sum(axis=1) + np.minimum(col[:,None],Y).sum(axis=0)

    swap_scores = u_scores + f_u_scores + np.minimum(G[u,:],H[mapping,f_u])+np.minimum(G[:,u],H[f_u,mapping])
    swap_scores -= scores_by_vertex
    swap_scores -= scores_by_vertex[u]
    swap_scores += np.minimum(G[u,:],H[f_u,mapping])+np.minimum(G[:,u],H[mapping,f_u])

    x = move_scores.argmax()
    w = swap_scores.argmax()
    if swap_scores[w] > 0 and swap_scores[w] >= move_scores[x]:

        f_w = mapping[w]

        scores_by_vertex += np.minimum(G[:,u],H[mapping,f_w])+np.minimum(G[u,:],H[f_w,mapping])+np.minimum(G[:,w],H[mapping,f_u])+np.minimum(G[w,:],H[f_u,mapping])
        scores_by_vertex -= np.minimum(G[:,u],H[mapping,f_u])+np.minimum(G[u,:],H[f_u,mapping])+np.minimum(G[:,w],H[mapping,f_w])+np.minimum(G[w,:],H[f_w,mapping])

        scores_by_vertex[u] = u_scores[w]+ np.minimum(G[u,w],H[f_w,f_u])+np.minimum(G[w,u],H[f_u,f_w])
        scores_by_vertex[w] = f_u_scores[w]+ np.minimum(G[u,w],H[f_w,f_u])+np.minimum(G[w,u],H[f_u,f_w])

        mapping[u] = f_w
        mapping[w] = f_u

        return swap_scores[w]
    elif move_scores[x] > 0:

        scores_by_vertex += np.minimum(G[:,u],H[mapping,x])+np.minimum(G[u,:],H[x,mapping]) - np.minimum(G[:,u],H[mapping,f_u])- np.minimum(G[u,:],H[f_u,mapping])
        
        scores_by_vertex[u] = base_scores[x]

        mapping[u] = x
        usage[f_u] -= 1
        usage[x] += 1

        return base_scores[x] - base_scores[f_u]
    else:
        return 0

In [54]:
G=B
H=A

top_score = score(G,H,mapping,require_surjective=False)
usage = np.zeros(mapping.shape,dtype=np.int32)
np.add.at(usage,mapping,1)
scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=scores_by_vertex.sum()//2

def penalty_gradient(usage):
   
   return 4*usage

for i in range(5):
  for j,u in enumerate(rng.permutation(N)):
    improvement = move_or_swap(G,H,mapping,u,scores_by_vertex,usage,penalty_gradient=penalty_gradient)
    score_track += improvement
    if j%20==0:
      print(f"{i=}, {j=}, {score_track=}, unique={len(np.unique(mapping))}",end="\r")

score(G,H,mapping,require_surjective=False), score(G,G,np.arange(N))

KeyboardInterrupt: 

In [53]:
scores_by_vertex.sum()//2,score_track, score(G,H,mapping,require_surjective=False)

(5751581, 5751581, 5751581)

In [42]:
len(np.unique(mapping))

15985